In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
data = pd.read_csv('./salary-train.csv')
data.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent,40000
3,A great local marketleader is seeking a perman...,Dereham,permanent,22500
4,Registered Nurse / RGN Nursing Home for Young...,Sutton Coldfield,NaN,20355


In [3]:
data['FullDescription'] = data.apply(lambda row : row['FullDescription'].lower(), axis = 1)

In [4]:
data['FullDescription'] = data['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(np.array(data['FullDescription']).tolist())

In [10]:
data['LocationNormalized'].fillna('nan', inplace=True)
data['ContractTime'].fillna('nan', inplace=True)

In [11]:
from sklearn.feature_extraction import DictVectorizer
enc = DictVectorizer()
X_train_categ = enc.fit_transform(data[['LocationNormalized', 'ContractTime']].to_dict('records'))

In [12]:
X_train_categ.todense()

matrix([[ 1.,  1.],
        [ 1.,  1.],
        [ 1.,  1.],
        ..., 
        [ 1.,  1.],
        [ 1.,  1.],
        [ 1.,  1.]])

In [13]:
from scipy.sparse import hstack
X = hstack([tfidf_matrix, X_train_categ])

In [14]:
from sklearn.linear_model import Ridge
clf = Ridge(random_state = 241, alpha = 1)
clf.fit(X, data.SalaryNormalized)

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=241, solver='auto', tol=0.001)

In [15]:
test = pd.read_csv('./salary-test-mini.csv')
test.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,We currently have a vacancy for an HR Project ...,Milton Keynes,contract,NaN
1,A Web developer opportunity has arisen with an...,Manchester,permanent,NaN


In [20]:
X_test_tfidf = tfidf.transform(test.FullDescription)
X_test_categ = enc.transform(test[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_test = hstack([X_test_tfidf, X_test_categ])

In [21]:
clf.predict(X_test)

array([ 57057.74260641,  38370.34131777])